In [ ]:
import sys
import os
import glob
import json
import numpy as np
import matplotlib.pyplot as plt

import cv2
# from google.colab import drive
# drive.mount('/content/drive')
# from google.colab.patches import cv2_imshow

import torch
from torch import nn, Tensor
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
from torchvision import datasets
from torchvision.io import read_image
import torchvision.transforms as tt

# !pip3 install rich
from rich.progress import Progress
from utils import get_abs_path, steer_to, lazy_states_contraction, is_feasible

In [ ]:
def cv2_imshow(img):
    img = np.concatenate((img[:,:,2:3], img[:,:,1:2], img[:,:,0:1]), axis=2)
    plt.imshow(img)
    plt.show()

In [ ]:
project_path = get_abs_path(1)

images_dir = project_path + '/data/maps/'
paths_dir = project_path + '/data/planned_maps/'
model_dir = project_path + '/models/'
enet_filename = model_dir + 'enet.pt'
pnet_filename = model_dir + 'pnet.pt'
paths_filename = paths_dir + 'paths.json'

train_enet = False
train_pnet = True

samples_count = 2000
training_percentage = 0.9
test_percentage = 0.1

training_count = int(samples_count * training_percentage)
test_count = int(samples_count * test_percentage)

In [ ]:
paths = {}
with open(paths_filename) as f:
  paths = json.load(f)

In [ ]:
class ImageDataset(Dataset):

    def __init__(self, images_dir, transform = None, target_transform = None):

        self.images_dir = images_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):

        files = []
        files.extend(glob.glob(self.images_dir + '*.png'))
        return len(files)

    def __getitem__(self, idx):

        image_path = self.images_dir + 'map_' + str(idx) + '.png'
        image = torch.Tensor(plt.imread(image_path))
        image = image.permute(2,1,0)
        # image = read_image(image_path)
        image = image[2, :, :]

        state = []
        if self.transform:
            state = torch.get_rng_state()
            image_in = self.transform(image)
        if self.target_transform:
            torch.set_rng_state(state)
            image_out = self.target_transform(image)

        return image, image

In [ ]:
encoder_data = ImageDataset(images_dir)

training_encoder_data, test_encoder_data = torch.utils.data.random_split(encoder_data, (training_count, test_count))

train_encoder_dataloader = DataLoader(training_encoder_data, batch_size=16, shuffle=True, pin_memory=True)
test_encoder_dataloader = DataLoader(test_encoder_data, batch_size=16, shuffle=True, pin_memory=True)

In [ ]:
def display_image(image):
    print(image.shape)
    image = image * 255
    image = image.permute(1, 2, 0).detach().numpy()
    cv2_imshow(image)

In [ ]:
train_features, train_labels = next(iter(train_encoder_dataloader))

display_image(train_features[0].unsqueeze(0))

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {0} device'.format(device))

In [ ]:
# def batch_norm_conv(ni, nf, stride=1):
#     return nn.Sequential(
#         nn.BatchNorm2d(ni),
#         nn.ReLU(inplace=True),
#         nn.Conv2d(ni, nf, (3, 3), stride=stride, padding=1, bias=False)
#     )

In [ ]:
# def conv1x1(ni, nf, stride=1):
#     return nn.Sequential(
#         nn.Conv2d(ni, nf, (1, 1), stride=stride),
#         nn.BatchNorm2d(nf)
#     )

In [ ]:
# class ResBlock(nn.Module):

#     def __init__(self, ni, nf, stride=1):
#         super(ResBlock, self).__init__()
#         self.shortcut = conv1x1(ni, nf, stride=stride)
#         self.Bn1 = batch_norm_conv(ni, nf, stride=stride)
#         self.Bn2 = batch_norm_conv(nf, nf)


#     def forward(self, x):
#         short = self.shortcut(x)
#         x = self.Bn1(x)
#         x = self.Bn2(x)
#         return x.add(short)

In [ ]:
# class ResEncoder(nn.Module):

#     def __init__(self):
#         super(ResEncoder, self).__init__()

#         self.short1 = conv1x1(1, 2)
#         self.conv1 = nn.Conv2d(1, 2, (3, 3), padding=1)
#         self.Bn1 = batch_norm_conv(2, 2)
#         self.Res1 = ResBlock(2, 4)
#         self.Res2 = ResBlock(4, 8, stride=2)
#         self.Res3 = ResBlock(8, 16, stride=2)
#         self.Res4 = ResBlock(16, 16, stride=2)


#     def forward(self, x):

#         r = self.short1(x)
#         x = self.conv1(x)
#         x = self.Bn1(x)
#         x = x.add(r)
#         short1 = torch.clone(x)

#         x = self.Res1(x)
#         short2 = torch.clone(x)
#         x = self.Res2(x)
#         short3 = torch.clone(x)
#         x = self.Res3(x)
#         short4 = torch.clone(x)
#         x = self.Res4(x)

#         return x, short1, short2, short3, short4

In [ ]:
# class ResDecoder(nn.Module):

#     def __init__(self):
#         super(ResDecoder, self).__init__()

#         self.Up1 = nn.UpsamplingBilinear2d(scale_factor=2)
#         self.Up2 = nn.UpsamplingBilinear2d(scale_factor=2)
#         self.Up3 = nn.UpsamplingBilinear2d(scale_factor=2)
#         self.Up4 = nn.UpsamplingBilinear2d(scale_factor=1)

#         self.Res5 = ResBlock(32, 8)
#         self.Res6 = ResBlock(16, 4)
#         self.Res7 = ResBlock(8, 2)
#         self.Res8 = ResBlock(4, 2)

#         self.conv2 = nn.Conv2d(2, 1, (1, 1))
#         self.sigmoid = nn.Sigmoid()


#     def forward(self, x, short1, short2, short3, short4):

#         x = self.Up1(x)
#         x = torch.cat((short4, x), dim=1)
#         x = self.Res5(x)

#         x = self.Up2(x)
#         x = torch.cat((short3, x), dim=1)
#         x = self.Res6(x)

#         x = self.Up3(x)
#         x = torch.cat((short2, x), dim=1)
#         x = self.Res7(x)

#         x = self.Up4(x)
#         x = torch.cat((short1, x), dim=1)
#         x = self.Res8(x)

#         x = self.conv2(x)
#         x = self.sigmoid(x)

#         return x

In [ ]:
# class ResUNet(nn.Module):

#     def __init__(self):
#         super(ResUNet, self).__init__()

#         self.Encoder = ResEncoder()
#         self.Decoder = ResDecoder()

#     def forward(self, x):

#         x, short1, short2, short3, short4 = self.Encoder(x)
#         x = self.Decoder(x, short1, short2, short3, short4)
#         return x

# Enet = ResUNet().to(device)
# # print(Enet.Encoder)
# # print(Enet.Decoder)

In [ ]:
class Encoder(nn.Module):
	def __init__(self):
		super(Encoder, self).__init__()
		
		self.flatten = nn.Flatten()
		self.dense1 = nn.Sequential(nn.Linear(14400, 512), nn.PReLU())
		self.dense2 = nn.Sequential(nn.Linear(512, 256), nn.PReLU())
		self.dense3 = nn.Sequential(nn.Linear(256, 128), nn.PReLU())
		self.dense4 = nn.Linear(128, 28)
			
	def forward(self, x):

		x = self.flatten(x)
		x = self.dense1(x)
		x = self.dense2(x)
		x = self.dense3(x)
		x = self.dense4(x)

		return x

In [ ]:

class Decoder(nn.Module):
	def __init__(self):
		super(Decoder, self).__init__()

		self.dense1 = nn.Sequential(nn.Linear(28, 128), nn.PReLU())
		self.dense2 = nn.Sequential(nn.Linear(128, 256), nn.PReLU())
		self.dense3 = nn.Sequential(nn.Linear(256, 512), nn.PReLU())
		self.dense4 = nn.Sequential(nn.Linear(512, 14400))
		self.unflatten = nn.Unflatten(1, (120, 120))

	def forward(self, x):
		
		x = self.dense1(x)
		x = self.dense2(x)
		x = self.dense3(x)
		x = self.dense4(x)
		x = self.unflatten(x)

		return x

In [ ]:
class ResUNet(nn.Module):

    def __init__(self):
        super(ResUNet, self).__init__()

        self.Encoder = Encoder()
        self.Decoder = Decoder()

    def forward(self, x):

        x = self.Encoder(x)
        x = self.Decoder(x)
        return x

Enet = ResUNet().to(device)
# print(Enet.Encoder)
# print(Enet.Decoder)

In [ ]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(Enet.parameters(), lr=1e-3)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):

    size = len(dataloader.dataset)
    model.train()

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f'Loss: {loss:>7f}  [{current:>5d}/{size:>5d}]')

In [ ]:
def test(dataloader, model, loss_fn):

    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            # correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    # correct /= size
    print(f'Avg loss: {test_loss:>8f} \n')

In [ ]:
if not os.path.exists(enet_filename) or train_enet == True:
    epochs = 100
    for t in range(epochs):
        print(f'Epoch {t+1}\n-------------------------------')
        train(train_encoder_dataloader, Enet, loss_fn, optimizer)
        test(test_encoder_dataloader, Enet, loss_fn)
    print('Done!')
    torch.save(Enet.state_dict(), enet_filename)
else:
    Enet.load_state_dict(torch.load(enet_filename))
    Enet.eval()

In [ ]:
test_features, test_labels = next(iter(test_encoder_dataloader))
display_image(test_features[0].unsqueeze(0))
display_image(test_labels[0].unsqueeze(0))

result = Enet(test_labels.to(device))
result = result.cpu()
display_image(result[0].unsqueeze(0))
print(Enet.Encoder(test_labels.to(device))[0].shape)

In [ ]:
class PathDataset(Dataset):

    def __init__(self, encoder, images_dir, paths):

        self.encoder = encoder
        self.images_dir = images_dir
        self.paths = paths


    def __len__(self):
        return len(self.paths)


    def __getitem__(self, idx):

        image_path = self.images_dir + 'map_' + str(idx) + '.png'
        image = Tensor(plt.imread(image_path)).to(device)
        image = image.permute(2,1,0)
        image = image[2:3, :, :]

        encoded_img = self.encoder(image.unsqueeze(0))[0].detach().cpu()

        # to replace bad path TODO fix
        path_len = 0
        i = idx
        while path_len < 2:
            map_name = 'map_' + str(i)
            planned_path = Tensor(self.paths[map_name])
            path_len = len(planned_path)
            i = np.random.randint(0, self.__len__())
        if path_len > 2:
            point_idx = np.random.randint(0, path_len - 2)
        else:
            point_idx = 0

        image_width = image.shape[1]
        x_start = planned_path[point_idx]
        x_goal = planned_path[-1]
        x_pred = planned_path[point_idx + 1]
        x_pred = x_pred

        # value normalization
        x_start /= image_width
        x_goal /= image_width
        x_pred /= image_width

        input_data = torch.cat([torch.flatten(encoded_img), x_start, x_goal])
        input_data.requires_grad = True
        
        return input_data, x_pred


dataset = PathDataset(Enet.Encoder, images_dir, paths)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True, pin_memory=True)
# TODO remove
# dataset[0][0].shape
# for i in range(len(dataset)):
#     print("#: ", i)
#     data, pred = dataset[i]
# np.random.randint(0, 1)

In [ ]:
# class PlanningNetwork(nn.Module):
#     def __init__(self, p=0.1):
#         super(PlanningNetwork, self).__init__()

#         enc_img_size = 3600
#         self.dense1 = nn.Sequential(nn.Linear(enc_img_size, 512), nn.PReLU(), nn.Dropout(p))
#         self.dense2 = nn.Sequential(nn.Linear(512, 256), nn.PReLU(), nn.Dropout(p))
#         self.dense3 = nn.Sequential(nn.Linear(256, 128), nn.PReLU(), nn.Dropout(p))
#         self.dense4 = nn.Sequential(nn.Linear(128, 64), nn.PReLU(), nn.Dropout(p))
#         self.dense5 = nn.Sequential(nn.Linear(64, 32), nn.PReLU(), nn.Dropout(p))
#         self.dense6 = nn.Sequential(nn.Linear(32, 16), nn.PReLU(), nn.Dropout(p))
#         self.dense7 = nn.Sequential(nn.Linear(16, 4), nn.PReLU(), nn.Dropout(p))
#         self.dense8 = nn.Sequential(nn.Linear(4, 2), nn.PReLU(), nn.Dropout(p))

#         self.dense9 = nn.Sequential(nn.Linear(4, 4), nn.PReLU(), nn.Dropout(p))
#         self.dense10 = nn.Sequential(nn.Linear(4, 2), nn.PReLU())
        
#     def forward(self, x):
        
#         c = x[:, -4:]
#         x = x[:, :-4]

#         x = self.dense1(x)
#         x = self.dense2(x)
#         x = self.dense3(x)
#         x = self.dense4(x)
#         x = self.dense5(x)
#         x = self.dense6(x)
#         x = self.dense7(x)
#         x = self.dense8(x)

#         c = self.dense9(c)
#         c = self.dense10(c)

#         x = torch.add(x, c)

#         return x

# Pnet = PlanningNetwork().to(device)
# print('Model parameters:', sum(p.numel() for p in Pnet.parameters() if p.requires_grad))

In [ ]:
# class PlanningNetwork(nn.Module):
#     def __init__(self, p=0.05):
#         super(PlanningNetwork, self).__init__()

#         enc_img_size = 3600

#         self.norm = nn.Sequential(nn.Linear(enc_img_size, 512), nn.PReLU(), nn.Dropout(p))

#         self.up_dense1 = nn.Sequential(nn.Linear(4, 8), nn.PReLU(), nn.Dropout(p))
#         self.up_dense2 = nn.Sequential(nn.Linear(8, 16), nn.PReLU(), nn.Dropout(p))
#         self.up_dense3 = nn.Sequential(nn.Linear(16, 32), nn.PReLU(), nn.Dropout(p))
#         self.up_dense4 = nn.Sequential(nn.Linear(32, 64), nn.PReLU(), nn.Dropout(p))
#         self.up_dense5 = nn.Sequential(nn.Linear(64, 128), nn.PReLU(), nn.Dropout(p))
#         self.up_dense6 = nn.Sequential(nn.Linear(128, 256), nn.PReLU(), nn.Dropout(p))
#         self.up_dense7 = nn.Sequential(nn.Linear(256, 512), nn.PReLU(), nn.Dropout(p))

#         self.dense1 = nn.Sequential(nn.Linear(1024, 512), nn.PReLU(), nn.Dropout(p))
#         self.dense2 = nn.Sequential(nn.Linear(512, 256), nn.PReLU(), nn.Dropout(p))
#         self.dense3 = nn.Sequential(nn.Linear(256, 128), nn.PReLU(), nn.Dropout(p))
#         self.dense4 = nn.Sequential(nn.Linear(128, 64), nn.PReLU(), nn.Dropout(p))
#         self.dense5 = nn.Sequential(nn.Linear(64, 32), nn.PReLU(), nn.Dropout(p))
#         self.dense6 = nn.Sequential(nn.Linear(32, 16), nn.PReLU(), nn.Dropout(p))
#         self.dense7 = nn.Sequential(nn.Linear(16, 8), nn.PReLU(), nn.Dropout(p))
#         self.dense8 = nn.Sequential(nn.Linear(8, 4), nn.PReLU(), nn.Dropout(p))
#         self.dense9 = nn.Sequential(nn.Linear(4, 2), nn.PReLU())
        
#     def forward(self, x):
        
#         c = x[:, -4:]
#         x = x[:, :-4]

#         x = self.norm(x)

#         c = self.up_dense1(c)
#         c = self.up_dense2(c)
#         c = self.up_dense3(c)
#         c = self.up_dense4(c)
#         c = self.up_dense5(c)
#         c = self.up_dense6(c)
#         c = self.up_dense7(c)

#         x = torch.cat([x ,c], dim=1)

#         x = self.dense1(x)
#         x = self.dense2(x)
#         x = self.dense3(x)
#         x = self.dense4(x)
#         x = self.dense5(x)
#         x = self.dense6(x)
#         x = self.dense7(x)
#         x = self.dense8(x)
#         x = self.dense9(x)

#         return x

# Pnet = PlanningNetwork().to(device)
# print('Model parameters:', sum(p.numel() for p in Pnet.parameters() if p.requires_grad))

In [ ]:
class PlanningNetwork(nn.Module):
    def __init__(self, p=0.5):
        super(PlanningNetwork, self).__init__()

        enc_img_size = 32
        self.dense1 = nn.Sequential(nn.Linear(enc_img_size, 1280), nn.PReLU(), nn.Dropout(p))
        self.dense2 = nn.Sequential(nn.Linear(1280, 1024), nn.PReLU(), nn.Dropout(p))
        self.dense3 = nn.Sequential(nn.Linear(1024, 896), nn.PReLU(), nn.Dropout(p))
        self.dense4 = nn.Sequential(nn.Linear(896, 768), nn.PReLU(), nn.Dropout(p))
        self.dense5 = nn.Sequential(nn.Linear(768, 512), nn.PReLU(), nn.Dropout(p))
        self.dense6 = nn.Sequential(nn.Linear(512, 384), nn.PReLU(), nn.Dropout(p))
        self.dense7 = nn.Sequential(nn.Linear(384, 256), nn.PReLU(), nn.Dropout(p))
        self.dense8 = nn.Sequential(nn.Linear(256, 128), nn.PReLU(), nn.Dropout(p))
        self.dense9 = nn.Sequential(nn.Linear(128, 64), nn.PReLU(), nn.Dropout(p))
        self.dense10 = nn.Sequential(nn.Linear(64, 32), nn.PReLU())
        self.dense11 = nn.Sequential(nn.Linear(32, 2))
        
    def forward(self, x):

        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        x = self.dense5(x)
        x = self.dense6(x)
        x = self.dense7(x)
        x = self.dense8(x)
        x = self.dense9(x)
        x = self.dense10(x)
        x = self.dense11(x)

        return x

Pnet = PlanningNetwork().to(device)
print('Model parameters:', sum(p.numel() for p in Pnet.parameters() if p.requires_grad))

In [ ]:
class Pnet_loss(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, output, target):

        print(output.shape, target.shape)

        loss = Tensor([0]).to(device)
        for i in range(len(target)):
            loss += torch.square(output[i, 0] - target[0][i, 0])
            loss += torch.square(output[i, 1] - target[0][i, 1])
        
        loss /= (target.shape[0] * 2)
        
        return loss

In [ ]:
loss_fn = Pnet_loss()
optimizer = torch.optim.Adagrad(Pnet.parameters(), lr=1e-4)

if not os.path.exists(pnet_filename) or train_pnet == True:
    epochs = 30
    for t in range(epochs):
        print(f'Epoch {t+1}\n-------------------------------')
        train(dataloader, Pnet, loss_fn.forward, optimizer)
        test(dataloader, Pnet, loss_fn.forward)
    print('Done!')
    torch.save(Pnet.state_dict(), pnet_filename)
else:
    Pnet.load_state_dict(torch.load(pnet_filename))
    Pnet.eval()

In [ ]:
# Display results
i = 110
image_path = images_dir + 'map_' + str(i) + '.png'
image = Tensor(plt.imread(image_path)).to(device)
image = image.permute(2,1,0)
image = image[2:3, :, :]

encoded_img = Enet.Encoder(image.unsqueeze(0))[0].detach().cpu()

planned_path = Tensor(paths['map_' + str(i)])
path_len = len(planned_path)

if path_len > 2:
    point_idx = np.random.randint(0, path_len - 2)
else:
    point_idx = 0

x_start = planned_path[point_idx] / image.shape[-1]
x_goal = planned_path[-1] / image.shape[-1]
x_pred = planned_path[point_idx + 1] / image.shape[-1]
input_data = torch.cat([torch.flatten(encoded_img), x_start, x_goal])

print(input_data.to(device).unsqueeze(0))

result = Pnet(input_data.to(device).unsqueeze(0))
result *= 120
result = torch.round(result)

plt.imshow(image.permute(1, 2, 0).detach().cpu().numpy())

print(x_start*120, x_goal*120, x_pred*120)
print(result.shape, x_pred.shape)
print(paths['map_' + str(i)])
print(result.tolist())

In [ ]:
def Replaning(a, b):
    pass

N = 10

In [ ]:
# Algorithm 1
def MPNet(x_init, x_goal, x_obs):
    Z = Enet.Encoder(x_obs)
    Tau = NeuralPlanner(x_init, x_goal, Z)
    if Tau:
        Tau = lazy_states_contraction(Tau)

        if is_feasible(Tau):
            return Tau
        else:
            Tau_new = Replaning(Tau, Z)
            Tau_new = lazy_states_contraction(Tau_new)
            if is_feasible(Tau_new):
                return Tau_new
    return 0

In [ ]:
# Algorithm 2
def NeuralPlanner(x_start, x_goal, Z):
    Tau_a = [x_start]
    Tau_b = [x_goal]
    Tau = 0

    for i in range(N):
        x_new = Pnet(Z, Tau_a[-1], Tau_b[-1])
        Tau_a.append(x_new)
        connectable = steer_to(Tau_a[-1], Tau_b[-1])

        if connectable:
            Tau = np.concatenate((Tau_a, Tau_b))
            return Tau

        Tau_a, Tau_b = Tau_b, Tau_a

    return 0